In [ ]:
import h5py
import numpy as np
import os
import time
import copy

from matplotlib import pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
sns.set_context('notebook', font_scale=1.1)

In [ ]:
notebook_start_time = time.time()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [ ]:
torch.manual_seed(1703) # we fix the random number generators here for the purpose of reproducibility
np.random.seed(1703)

## Problem description

In this tutorial, we want to use supervised learning for establishing a neural network that can predict wind speed from CyGNSS observations (see lecture). Since wind speed is a continuous variable, this is a *regression* problem. We have prepared training, validation, and test data. 

Your task is to find the best possible neural network for predicting wind speed. We will walk you through various choices you can make to the data and the model algorithm. You will train various models and compare their performance on the validation dataset. Finally, you will choose your best model and make predictions on the test dataset.

A sample consists of *input features* and a *target label*. One such sample is shown below:

In [ ]:
from IPython.display import Image
Image('./demo_sample.png', height=1)

## Setup

Training a neural network in pytorch requires setting up several components:
- data set: makes your raw data set accessible to pytorch
- data loader: fetches data in batches
- neural network architecture
- method for training the network
- method for making predictions with the network
    
For the purpose of this tutorial, it is not necessary to understand all these components in detail, however, they are documented and explained for later reference. You may execute the following cells up to "Interactive tutorial" and start with exploring the tasks.

### Data set

The pytorch dataset and data loader are discussed in more detail in the workshop on data processing.

In [ ]:
class CyGNSSDataset(Dataset):
    def __init__(self, flag, input_v_map=['brcs'], normalization_values=None, filter_quality=False):
        '''
        Load data and apply transforms during setup

        Parameters:
        -----------
        flag : string
            Any of train / valid / test. Defines dataset.
        input_v_map : list
            Input maps, choice of ['brcs', 'eff_scatter']
        normalization_values : dict
            Mean and standard deviation, needed for scaling the input variables
        filter_quality : bool
            Filter samples that are flagged as bad quality (default: False)
        -----------
        Returns: dataset
        '''
        self.h5_file = h5py.File(os.path.join('/work/ka1176/shared_data/CyGNSS/', flag + '_data.h5'), 'r', rdcc_nbytes=0)  # disable cache
        # load everything into memory
        start_time = time.time()
        
        # load labels
        self.y = self.h5_file['windspeed'][:].astype(np.float32)

        # normalize main input data
        # Save normalization values together with the trained model
        # For inference load the normalization values

        if flag=='train': # determine normalization values
            self.normalization_values = dict()
        else:
            self.normalization_values = normalization_values
        
        # stack map vars (2D vars)
        self.X = []
        for v_map in input_v_map:
            X_v_map = self.h5_file[v_map][:].astype(np.float32)
            
            if flag=='train':
                norm_vals = dict()
                X_v_map_scaled, X_mean, X_std = self._standard_scale(X_v_map)
                self.normalization_values[f'{v_map}_mean'] = X_mean
                self.normalization_values[f'{v_map}_std']  = X_std
            else:
                X_mean = self.normalization_values[f'{v_map}_mean']
                X_std = self.normalization_values[f'{v_map}_std']
                X_v_map_scaled = self._standard_scale_given(X_v_map, X_mean, X_std)
                
            self.X.append(X_v_map_scaled) # append scaled 2D map
            #self.X.append(X_v_map) # append unscaled 2D map (test)
        self.X = np.stack(self.X, axis=1)
        
        if filter_quality:
            n_before = len(self.y)
            mask = self.h5_file['quality'][:]
            self.X, self.y = self.X[mask], self.y[mask]
            print(f'After filter_quality, {len(self.y)} samples remain ({len(self.y)/n_before*100:.1f}%)')

        print(f'load and transform {flag} input data: {self.X.shape} ({self.X.nbytes // 1e6}MB)')
        print(f'load and transform {flag} labels: {self.y.shape} ({self.y.nbytes // 1e6}MB)')
        
    def _standard_scale(self, v):
        '''apply standard scale and return mean / std'''
        mean = np.mean(v)
        sigma = np.std(v)
        v_tilde = (v - mean) / sigma
        return v_tilde, mean, sigma
    
    def _standard_scale_given(self, v, mean, sigma):
        '''apply standard scale with pre-determined mean / std'''
        v_tilde = (v - mean) / sigma
        return v_tilde

    def _filter_all_data_by_mask(self, mask, flag, name=''): 
        '''filter the input data by the provided mask'''
        self.X, self.y = self.X[mask], self.y[mask]
        print(f'{flag} input data after {name} downsampling: {self.X.shape} ({self.X.nbytes // 1e6}MB)')

    def __len__(self):
        '''required function for the pytorch dataloader'''
        return self.X.shape[0]

    def __getitem__(self, idx):
        '''required function for the pytorch dataloader'''
        X = self.X[idx]
        y = self.y[idx]
        return (X, y)


### Data loader

In [ ]:
def setup_dataloaders(filter_quality=False, input_v_map=['brcs']):
    '''Load the datasets and create PyTorch dataloaders
    
    Input parameters:
    -------------------------
    filter_quality : apply a filter for sample quality (default: False)
    input_v_map    : list of input features (default: ['brcs'])
    -------------------------
    
    Returns:
    -------------------------
    pytorch DataLoader instances for train / validation / test set
    '''
    
    train_dataset = CyGNSSDataset('train', filter_quality=filter_quality, input_v_map=input_v_map)
    valid_dataset = CyGNSSDataset('valid', filter_quality=filter_quality, input_v_map=input_v_map, normalization_values=train_dataset.normalization_values)
    test_dataset = CyGNSSDataset('test', filter_quality=filter_quality, input_v_map=input_v_map, normalization_values=train_dataset.normalization_values)
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    test_dataloader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    
    return train_dataloader, valid_dataloader, test_dataloader

### Neural network architecture

We pre-defined two network architectures for this tutorial:
- Fully connected network
- Convolutional neural network

At initialization, the individual network components are defined (layers like nn.Linear, nn.Dropout, nn.Conv2d). In the forward function, the forward pass through the network is defined, i.e., the order in which the layers are applied to the input.

In [ ]:
class FullyConnectedNetwork(nn.Module):
    '''A fully connected neural network
    
    Input parameters:
    ----------------------
    units_dense1 : first layer dimension
    units_dense2 : second layer dimension
    dropout      : dropout value following the second layer
    '''
    def __init__(self, units_dense1=64, units_dense2=32, dropout=0, input_shape=(1, 17, 11)):
        super(FullyConnectedNetwork, self).__init__()
        # first dense layer
        self.fc1 = nn.Linear(np.product(input_shape), units_dense1)
        # second dense layer
        self.fc2 = nn.Linear(units_dense1, units_dense2)
        # dropout layer for regularization
        self.dr_fc2 = nn.Dropout(dropout)
        # final layer producing the output
        self.fc_final = nn.Linear(units_dense2, 1)

    def forward(self, x):
        # flatten the input images to 1D arrays
        x = torch.flatten(x, 1)
        # first layer
        x = self.fc1(x)
        # activation function
        x = F.relu(x)
        # second layer
        x = self.fc2(x)
        # activation function
        x = F.relu(x)
        # dropout layer
        x = self.dr_fc2(x)
        # output layer
        x = self.fc_final(x)
        return x

In [ ]:
class ConvolutionalNeuralNetwork(nn.Module):
    '''A convolutional neural network with 2 convolutional layers, followed by two dense layers'''
    def __init__(self, filters=16, units_dense1=32, units_dense2=16, dropout=0, input_shape=(1, 17, 11)):
        super(ConvolutionalNeuralNetwork, self).__init__()
        
        self.cv1 = nn.Conv2d(input_shape[0], filters, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(filters)
        self.pl1 = nn.MaxPool2d(2)
        self.cv2 = nn.Conv2d(filters, 2*filters, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(2*filters)
        
        S = int(filters * 2 * 5 * 8)
        
        self.fc1 = nn.Linear(S, units_dense1)
        self.fc2 = nn.Linear(units_dense1, units_dense2)
        self.dr_fc2 = nn.Dropout(dropout)
        self.fc_final = nn.Linear(units_dense2, 1)

    def forward(self, x):
        
        x = F.relu(self.cv1(x))
        x = self.bn1(x)
        x = self.pl1(x)
        x = F.relu(self.cv2(x))
        x = self.bn2(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dr_fc2(F.relu(self.fc2(x)))
        x = self.fc_final(x)
        return x

### Training procedure

In [ ]:
def train_model(model, train_dataloader, valid_dataloader, max_epochs=100, early_stopping=False, patience=3, learning_rate=1e-3, verbose=True):
    '''
    Train a model.
    
    model : the model to train
    train_dataloader : the dataloader for the train dataset
    valid_dataloader : the dataloader for the validation dataset
    max_epochs : maximum number of epochs to train (default: 100)
    early_stopping : apply the early stopping condition (default: False)
    patience : how many epochs to wait for early stopping (default: 3)
    learning_rate : optimizer learning rate (default: 0.001)
    verbose : print losses after each epoch (default: True)
    
    Returns:
    model : the trained model
    train_losses : history of train loss per epoch 
    valid_losses : history of validation loss per epoch
    '''
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    train_losses = []
    valid_losses = []
    
    if early_stopping:
        best_valid_loss = np.inf
        patience_counter = 0
    
    start_time = time.time()
    
    for epoch in range(max_epochs):
        # train
        model.train()
        epoch_train_loss = []
        for batch_idx, (features, target) in enumerate(train_dataloader):
            optimizer.zero_grad()
            output = model(features)
            output = torch.squeeze(output, dim=1)
            loss = F.mse_loss(output, target)
            loss.backward()
            optimizer.step()
            epoch_train_loss.append(loss.item())
        
        tl = np.mean(epoch_train_loss)
        
        # validate
        model.eval()
        epoch_valid_loss = []
        for batch_idx, (features, target) in enumerate(valid_dataloader):
            output = torch.squeeze(model(features), dim=1)
            loss = F.mse_loss(output, target)
            epoch_valid_loss.append(loss.item())
        vl = np.mean(epoch_valid_loss)
        
        # save losses for training history
        train_losses.append(tl)
        valid_losses.append(vl)
        if verbose:
            print(f'Epoch {epoch}: train loss = {tl:.4f}, valid loss = {vl:.4f}')
        
        if early_stopping:
            if vl < best_valid_loss:
                best_valid_loss = vl
                best_model = copy.deepcopy(model)
                patience_counter = 0
            else:
                patience_counter += 1
            if patience_counter >= patience:
                print(f'Applying early stopping condition after validation loss did not improve for {patience} epochs.')
                model = copy.deepcopy(best_model) # re-instate the best model
                break

    print(f'Finished training in {epoch+1} epochs and {time.time()-start_time:.1f} seconds.')
        
    return model, train_losses, valid_losses

### Prediction procedure

In [ ]:
def predict(model, dataloader, include_plot=False):
    '''Make and return predictions with a trained model
    
    Input parameters:
    ------------------------
    model : trained model
    dataloader : dataloader for the dataset that we want to predict on (typically: validation during evaluation / test for the final predictions)
    include_plot : generate a 2D histogram plot (default: False)
    
    
    Returns:
    ------------------------
    Predictions as a 1D numpy array
    '''
    model.eval()
    predict_loss = []
    outputs = []
    weights = []
    for batch_idx, (features, target) in enumerate(dataloader):
            output = torch.squeeze(model(features), dim=1)
            loss = F.mse_loss(output, target)
            predict_loss.append(loss.item())
            outputs.append(output.detach().numpy())
            weights.append(len(target) / batch_size)
    mse = np.average(predict_loss, weights=weights)
    print(f'Prediction: Loss = {mse:.4f}')
    print(f'--> RMSE = {np.sqrt(mse):.2f} m/s')
    
    y_pred = np.concatenate(outputs)
    
    if include_plot:
        plt.hexbin(dataloader.dataset.y[:len(y_pred)], y_pred, mincnt=1, cmap='viridis')
        plt.colorbar(label='Sample count')
        ax=plt.gca()
        ax.set_aspect('equal')
        ax.set_xlabel('True wind speed (m/s)')
        ax.set_ylabel('Predicted wind speed (m/s)')
        ax.plot(range(0, 20), range(0, 20), 'r--')
        plt.show()
    
    return y_pred

## Interactive tutorial

### Network and training parameters

Network and training parameters need to be determined individually for each machine learning problem. For this tutorial, we selected reasonable values:

In [ ]:
batch_size = 128      # number of samples in one mini-batch of training data
learning_rate = 1e-3  # ADAM optimizer learning rate
max_epochs = 75       # maximum number of training epochs

### First model

Create the dataset and dataloaders

In [ ]:
train_dataloader, valid_dataloader, test_dataloader = setup_dataloaders()

Train the first model. This may take few minutes.

In [ ]:
model = FullyConnectedNetwork()
print(model)

In [ ]:
model, train_losses, valid_losses = train_model(model, train_dataloader, valid_dataloader, max_epochs=max_epochs, learning_rate=learning_rate)

### Analyse the training process

During training, we recorded the loss each epoch on the training and the validation set. 

By executing the following cell, plot the losses as a function of epoch. We see as the training progresses, the loss on the training set decreases (bias is reduced). However, the performance on the validation set does not improve further at some point. Even worse, the validation loss may start to grow again.

TASK: Try this out by changing the parameter may_epochs to a larger value, e.g., 150. 

In [ ]:
plt.plot(train_losses, 'o:', label='Train')
plt.plot(valid_losses, 'o:', label='Valid')
plt.ylim(2, 5.5)
plt.xlabel('Epoch')
plt.ylabel('MSE (Loss)')
plt.legend(loc=1)
plt.show()

### Early stopping

One strategy to combat overfitting is to employ an Early Stopping condition. We monitor the loss on the validation set, and once it ceases to improve, we interrupt the training process.

TASK: With the following cells, train the model again, this time employing the Early Stopping condition. For how many epochs did the network train?

In [ ]:
model = FullyConnectedNetwork()
model, train_losses, valid_losses = train_model(model, train_dataloader, valid_dataloader, max_epochs=max_epochs, learning_rate=learning_rate, early_stopping=True)

In [ ]:
plt.plot(train_losses, 'o:', label='Train')
plt.plot(valid_losses, 'o:', label='Valid')
plt.ylim(2, 5.5)
plt.xlabel('Epoch')
plt.ylabel('MSE (Loss)')
plt.legend(loc=1)
plt.show()

Display the predictions and the true values. Ideally, these would lie on the red diagonal line. The root mean squared error (RMSE) is the quantity we use for evaluation. We train different models and compare the RMSE on the validation set to find the optimal one.

In [ ]:
y_pred_valid = predict(model, valid_dataloader, include_plot=True)

## Advanced topics

We can improve the predictions by improving the model and/or improving the data

### Change the model capacity

TASK: Experiment with the dimension of the hidden layer by adjusting the values of units_dense1, units_dense2 in the following cell. The default values are units_dense1=64, units_dense2=32 (you may check the definition of the FullyConnectedNetwork above to verify that). 

As a starting point, try these two combinations:
- units_dense1 = 4, units_dense2 = 2
- untis_dense1 = 256, units_dense2 = 128

How does the dimension of the hidden layer affect the validation loss? Can you find other promising combinations of units_dense1, units_dense2?

In [ ]:
units_dense1 = 64 # change this parameter
units_dense2 = 32 # change this parameter

In [ ]:
model = FullyConnectedNetwork(units_dense1=units_dense1, units_dense2=units_dense2)
model, train_losses, valid_losses = train_model(model, train_dataloader, valid_dataloader, max_epochs, early_stopping=True, learning_rate=learning_rate, verbose=False)

In [ ]:
y_pred_valid = predict(model, valid_dataloader, include_plot=True)

### Add the quality filter to the data set

There are some samples in the data set that ended up having the wrong label. This can happen quite often, and it is one of the main challenges of data processing to improve the dataset quality. In our dataset, there are some samples that have been labeled with the wrong wind speeds. We identified these samples and introduced a mask that we can now use for filtering the samples. More details will be given in Part 2 of our ML workshop.

In the next cell, the dataset is reloaded, this time using the flag "filter_quality=True". 

TASK: Reload the dataset and train the model again. Display the network predictions and compare them to the predictions from before. What do you observe?

In [ ]:
train_dataloader, valid_dataloader, test_dataloader = setup_dataloaders(filter_quality=True)

In [ ]:
model = FullyConnectedNetwork(units_dense1=64, units_dense2=32) # you may use better performing parameters from the last exercise
model, train_losses, valid_losses = train_model(model, train_dataloader, valid_dataloader, max_epochs, early_stopping=True, learning_rate=learning_rate, verbose=False)

In [ ]:
y_pred_valid = predict(model, valid_dataloader, include_plot=True)

### Add another input feature

The CyGNSS dataset offers several input features. So far, we used only the feature 'brcs' (the bi-static radar cross section). Now, we add a second input feature 'eff_scatter' (the effective scatter area map), based on the suggestion of a domain scientist. 

In the data set, we have one more type of input data available. So far, we restricted ourselves to the feature "brcs", but now we add the second feature "eff_scatter". We are still working with a fully connected network, all input data is flattened and concatenated:

Input shape (batch_size, 2, 17, 11) --> (batch_size, 1, 2 * 17 * 11) = (batch_size, 1, 374)

TASK: Add the second input feature below. Execute the cells to repeat data set creation and model training. What is the effect on the validation set performance?

In [ ]:
input_v_map = ['brcs'] # change this to include eff_scatter as well

train_dataloader, valid_dataloader, test_dataloader = setup_dataloaders(filter_quality=True, input_v_map=input_v_map)

In [ ]:
model = FullyConnectedNetwork(units_dense1=units_dense1, units_dense2=units_dense2, input_shape=(2,17,11))
model, train_losses, valid_losses = train_model(model, train_dataloader, valid_dataloader, max_epochs, early_stopping=True, learning_rate=learning_rate, verbose=False)

In [ ]:
y_pred_valid = predict(model, valid_dataloader, include_plot=True)

### Run CNN instead of fully connected network

Since we are dealing with 2-dimensional input features that look a lot like images, we would like to use a convolutional neural network (CNN). These networks can exploit the relationships between neighboring pixels and can identify and extract relevant features. The number of filters in the convolutional neural network we defined for you can be changed with the parameter filters.

TASK: Use the following cells to train a convolutional neural network. Experiment with the number of filters. 

How does the convolutional neural network perform compared to the fully connected neural network from before? 

How does the number of filters affect the validation set performance?

In [ ]:
filters = 32 # change this value

In [ ]:
model = ConvolutionalNeuralNetwork(input_shape=(2,17,11), filters=filters)
model, train_losses, valid_losses = train_model(model, train_dataloader, valid_dataloader, max_epochs, early_stopping=True, learning_rate=learning_rate, verbose=True)

In [ ]:
y_pred_valid = predict(model, valid_dataloader, include_plot=True)

## Evaluate the model on the test set

TASK: Finally, choose the model and training parameters that produced the best results on the validation set. Train the model, and use it to make predictions on the test set. Is the loss on the test set comparable to the loss on the validation set?

In [ ]:
model = ... # add your model definition here
model, train_losses, valid_losses = train_model(model, train_dataloader, valid_dataloader, max_epochs, early_stopping=True, learning_rate=learning_rate, verbose=True)

In [ ]:
y_pred = predict(model, test_dataloader, include_plot=True)

In [ ]:
print(f'Running the notebook took {(time.time()-notebook_start_time)/60} minutes')